In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import classification_report, accuracy_score, make_scorer


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline


import math
import bert
from tensorflow.keras import layers
from transformers import AutoModel, AutoTokenizer
import re
import random

In [2]:
column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")
df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [3]:
df.loc[df.duygu=="olumlu","duygu"]=1
df.loc[df.duygu=="olumsuz","duygu"]=0
df[:4]

,tweets,duygu
0,turkcell heryerde çekiyor kesin bilgi,1
1,turkcell olmak ayrıcalıktir çünkü kuzenlerin v...,1
2,allahtan turkcell'liyim amin,1
3,avea kaşar yaşasın turkcell,1


In [4]:
df=df.sample(frac=1).reset_index(drop=True)
df

,tweets,duygu
0,turkcell taşlara gelesin yaw ne oldu bu turkce...,0
1,ay turkcell bana bedava internet vermis yerim ...,1
2,bizde kaç yıllık turkcell hattımızı vodafona t...,0
3,"turkcell , vodafone, avea boşuna reklam yapıyo...",0
4,en başarılı hopi kampanyası turkcell in derim ...,1
...,...,...
9121,50 mbps bant genişliği olan turkcell superonli...,1
9122,bu turkcell 'e numaranızı taşımayın.. hele söz...,0
9123,ama ben senin 4 gb internetini yeğğğrim turkcell,1
9124,abv turkcell iligimi kemigimi kuruttun,0


In [5]:
tweets = list(df['tweets'])


In [6]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased",do_lower_case=True)

In [7]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [8]:
#cnn
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=256,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.3,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=5,
                                        padding="same",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        
        concatenated = tf.concat([l_1], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [9]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2
DROPOUT_RATE = 0.2
NB_EPOCHS = 50
early_stopping = EarlyStopping(monitor='loss', patience=3, verbose=True)

In [10]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [11]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif


In [12]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
BATCH_SIZE = 16

for train_index, test_index in kfold.split(df):
    # splitting Dataframe (dataset not included)
    
    train_df = df.iloc[train_index].tweets.tolist()
    test_df = df.iloc[test_index].tweets.tolist()
    train_y = np.array(df.iloc[train_index].duygu.tolist())
    test_y = np.array(df.iloc[test_index].duygu.tolist())

    tokenized_reviews_train = [tokenize_reviews(tweet) for tweet in train_df]
    tokenized_reviews_test = [tokenize_reviews(tweet) for tweet in test_df] 
    
    reviews_with_len_train = [[review, train_y[i], len(review)]
                     for i, review in enumerate(tokenized_reviews_train)]
    
    reviews_with_len_test = [[review, test_y[i], len(review)]
                     for i, review in enumerate(tokenized_reviews_test)]

    random.shuffle(reviews_with_len_train)
    random.shuffle(reviews_with_len_test)
    
    reviews_with_len_train.sort(key=lambda x: x[2])
    reviews_with_len_test.sort(key=lambda x: x[2])
    
    sorted_reviews_train = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len_train]
    sorted_reviews_test = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len_test]
    
    processed_dataset_test = tf.data.Dataset.from_generator(lambda: sorted_reviews_test, output_types=(tf.int32, tf.int32))
    processed_dataset_train = tf.data.Dataset.from_generator(lambda: sorted_reviews_train, output_types=(tf.int32, tf.int32))

    test_data = processed_dataset_test.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))
    train_data = processed_dataset_train.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    text_model.fit(train_data, epochs=50,callbacks=early_stopping)
    
    loss, accuracy, precision, recall = text_model.evaluate(test_data)

    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    # Increase fold number
    fold_no = fold_no + 1

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
514/514 [==============================] - 54s 106ms/step - loss: 0.4570 - acc: 0.7761 - precision: 0.7550 - recall: 0.8197.4600 - acc: 0.7741 - prec
Epoch 2/50
514/514 [==============================] - 109s 211ms/step - loss: 0.1574 - acc: 0.9401 - precision: 0.9407 - recall: 0.9398
Epoch 3/50
514/514 [==============================] - 114s 221ms/step - loss: 0.0451 - acc: 0.9861 - precision: 0.9848 - recall: 0.9876
Epoch 4/50
514/514 [==============================] - 113s 219ms/step - loss: 0.0159 - acc: 0.9951 - precision: 0.9956 - recall: 0.9947
Epoch 5/50
514/514 [==============================] - 113s 220ms/step - loss: 0.0257 - acc: 0.9916 - precision: 0.9915 - recall: 0.9917
Epoch 6/50
514/514 [==============================] - 114s 222ms/step - loss: 0.0192 - acc: 0.9938 - precision: 0.9942 - recall: 0.9934 - loss: 0.0181 - acc: 0.9945 
Epoch 7/50
514/514 [============

514/514 [==============================] - 107s 209ms/step - loss: 4.0558e-04 - acc: 0.9996 - precision: 0.9995 - recall: 0.9998
Epoch 00007: early stopping
58/58 [==============================] - 1s 18ms/step - loss: 0.0021 - acc: 0.9989 - precision: 1.0000 - recall: 0.9978
------------------------------------------------------------------------
Training for fold 5 ...
Epoch 1/50
514/514 [==============================] - 104s 202ms/step - loss: 0.0054 - acc: 0.9988 - precision: 0.9990 - recall: 0.9985
Epoch 2/50
514/514 [==============================] - 106s 207ms/step - loss: 0.0065 - acc: 0.9982 - precision: 0.9978 - recall: 0.9985
Epoch 3/50
514/514 [==============================] - 107s 208ms/step - loss: 0.0076 - acc: 0.9985 - precision: 0.9988 - recall: 0.9983
Epoch 4/50
514/514 [==============================] - 104s 202ms/step - loss: 5.7344e-04 - acc: 0.9998 - precision: 0.9998 - recall: 0.999836s - loss: 8.3690e-04 - acc: 0.9996 - precision: 0.999 - ETA: 32s - loss: 7.95

514/514 [==============================] - 73s 142ms/step - loss: 2.2170e-07 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000s - loss: 2.2528e-07 - acc: 1.0000 - precision:
Epoch 18/50
514/514 [==============================] - 52s 101ms/step - loss: 1.3416e-05 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 19/50
514/514 [==============================] - 68s 132ms/step - loss: 2.7465e-07 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 20/50
514/514 [==============================] - 74s 145ms/step - loss: 2.4673e-08 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 21/50
514/514 [==============================] - 73s 142ms/step - loss: 6.1848e-08 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 22/50
514/514 [==============================] - 74s 144ms/step - loss: 1.1538e-08 - acc: 1.0000 - precision: 1.0000 - recall: 1.0000
Epoch 23/50
514/514 [==============================] - 72s 139ms/step - loss: 0.0079 - acc: 0.9987 - precision: 0.9985 - reca

In [13]:
def Average(lst):
    return sum(lst) / len(lst)

In [14]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.981598
test precision: 0.980711
test recall: 0.981852
test f1_score: 0.981275
